<a href="https://colab.research.google.com/github/amitchug/ALMlops/blob/main/M3_AST_02_TextPreprocessing_using_NLTK_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Programme in AI and MLOps
## A Program by IISc and TalentSprint
### Assignment 2: Text Preprocessing using NLTK

## Learning Objectives

At the end of the experiment, you will be able to:

* understand the use of NLTK library
* perform text pre-processing using NLTK such as removing html strips and noise text, removing special characters, lemmatization, stemming, tokenization, removing stop words

## Introduction

**NLTK** (Natural Language Toolkit) is a leading platform for building Python programs to work with human language data. It provides easy-to-use interfaces to over 50 corpora and lexical resources such as WordNet, along with a suite of text processing libraries for classification, tokenization, stemming, tagging, parsing, and semantic reasoning, wrappers for industrial-strength NLP libraries.

It is a free, open source, community-driven project.

## Dataset Description

The IMDB movie review dataset can be downloaded from [here](http://ai.stanford.edu/~amaas/data/sentiment/). This dataset for binary sentiment classification contains around 50k movie reviews with the following attributes:

* **review:** text based review of each movie
* **sentiment:** positive or negative sentiment value


### Setup Steps:

In [ ]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "" #@param {type:"string"}

In [ ]:
#@title Please enter your password (your registered phone number) to continue: { run: "auto", display-mode: "form" }
password = "" #@param {type:"string"}

In [ ]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython

ipython = get_ipython()

notebook= "M3_AST_02_TextPreprocessing_using_NLTK_A" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")
    ipython.magic("sx wget https://cdn.iisc.talentsprint.com/DLFA/Experiment_related_data/IMDB_Dataset.csv")
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")

    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:
        print(r["err"])
        return None
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional,
              "concepts" : Concepts, "record_id" : submission_id,
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook,
              "feedback_experiments_input" : Comments,
              "feedback_mentor_support": Mentor_support}
      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:
        print(r["err"])
        return None
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://aimlops-iisc.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id


def getAdditional():
  try:
    if not Additional:
      raise NameError
    else:
      return Additional
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None

def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None


# def getWalkthrough():
#   try:
#     if not Walkthrough:
#       raise NameError
#     else:
#       return Walkthrough
#   except NameError:
#     print ("Please answer Walkthrough Question")
#     return None

def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None


def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer():
  try:
    if not Answer:
      raise NameError
    else:
      return Answer
  except NameError:
    print ("Please answer Question")
    return None


def getId():
  try:
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
else:
  print ("Please complete Id and Password cells before running setup")



Setup completed successfully


### Importing required packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk                                                                        # platform for building Python programs to process natural language
nltk.download('stopwords')                                                         # to download the stop words
nltk.download('punkt')                                                             # tokenizer divides a text into a list of sentences, by using an unsupervised algorithm to build a model for abbreviation words, collocations, and words that start sentences
nltk.download('wordnet')                                                           # to lemmatize word using WordNet's built-in function
nltk.download('averaged_perceptron_tagger')                                        # for part-of-speech tagger
from nltk.corpus import stopwords                                                  # importing the NTLK stopwords to remove articles, preposition and other words that are not actionable
from nltk.stem.porter import PorterStemmer                                         # process of reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words
from nltk.tokenize import word_tokenize                                            # allows to create individual objects from a bag of words
from bs4 import BeautifulSoup                                                      # Python library for pulling data from HTML and XML files
import re                                                                          # regular expression (or RE) specifies a set of strings that matches it
import string
import warnings
warnings.filterwarnings('ignore')

### Load the dataset

In [ ]:
# read the dataset
df = # YOUR CODE HERE to read 'IMDB_Dataset.csv'
print(df.shape)
df.head(10)      # first 10 rows

In [ ]:
# Let us view one of the reviews
# YOUR CODE HERE

### Exploratory Data Analysis

In [ ]:
# summary of the dataset
# YOUR CODE HERE

Now, we will look at the sentiment count by category.

In [ ]:
# sentiment count category-wise
# YOUR CODE HERE

In [ ]:
# Visualize the postive and negative sentiments in a bar plot
# YOUR CODE HERE

We can see that the dataset is balanced.

Now, we will do the text cleaning of the reviews.



## Text Preprocessing

The data scraped from the website is mostly in the raw text form. This data needs to be cleaned before analyzing it or fitting a model to it. Cleaning up the text data is necessary to highlight the attributes that we are going to want our machine learning system to pick up on.

**Removing noisy text**

Sample noise removal tasks could include:

* removing text file headers, footers
* removing HTML, XML, etc. markup and metadata
* extracting valuable data from other formats, such as JSON

In [ ]:
# removing the html strips
def strip_html(text):
    # BeautifulSoup is a useful library for extracting data from HTML and XML documents
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

# removing the square brackets
def remove_between_square_brackets(text):
    text = text.replace('[', '')
    # YOUR CODE HERE to remove ']'
    return text
    #return re.sub('\[[^]]*\]', '', text)          # to remove the text also present within square brackets

# removing the noisy text
def denoise_text(text):
    # YOUR CODE HERE to update the text by applying strip_html() function
    # YOUR CODE HERE to update the text by applying remove_between_square_brackets() function
    return text

In [ ]:
txt = """<body>
<h1>This is a test heading.</h1>
<p>Starting a paragraph here...</p>
</body>"""

# YOUR CODE HERE to call strip_html() on txt

In [ ]:
txt = "I bought [apple, banana, orange] yesterday."

# YOUR CODE HERE to call remove_between_square_brackets() on txt

In [ ]:
# apply denoise_text() function on review column
# YOUR CODE HERE

**Removing special characters**

Special characters typically include any character that is not a letter or number, such as punctuation. Removing special characters from a string results in a string containing only letters and numbers.

We can use the `re` python library for Regular expression operations.

To know more about Regular expressions, click [here](https://realpython.com/regex-python/).

In [ ]:
# define function for removing special characters
def remove_special_characters(text):
    pattern = r'[^a-zA-Z0-9\s]'
    text = re.sub(pattern, '', text)
    return text

In [ ]:
txt = "Hi There! How are you?"

# YOUR CODE HERE to call remove_special_characters() on txt

In [ ]:
# apply remove_special_characters() function on review column
# YOUR CODE HERE

**Lemmatization**

Lemmatization is a text pre-processing technique used to break a word down to its root meaning or word (called lemma) to identify similarities.

For example, a lemmatization algorithm would reduce the word ***better*** to its root word, or lemme, ***good***.

In [ ]:
# Lemmatize word using WordNet's built-in function
# pos: The Part Of Speech tag.
#      Valid options are "n" for nouns, "v" for verbs, "a" for adjectives,
#                        "r" for adverbs and "s" for satellite adjectives.

lemmatizer = nltk.stem.WordNetLemmatizer()
print("better :", lemmatizer.lemmatize("better", pos ="a", ))

**Text Stemming**

Stemming, also called suffix stripping, is a technique used to reduce text dimensionality. Stemming is also a type of text normalization that enables you to standardize some words into specific expressions also called stems.

In [ ]:
# stemming the text
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

In [ ]:
txt = "He likes lemons bananas and oranges. He goes to market."

# YOUR CODE HERE to call simple_stemmer() on txt

In [ ]:
# apply function on review column
# YOUR CODE HERE to apply 'simple_stemmer' function on reviews

**Tokenization**

Tokenization is the process of splitting paragraphs and sentences into smaller understandable parts (words).

For example:

In [ ]:
tokens = word_tokenize('I enjoy playing football in the rain.')
tokens

**Part-of-speech tagging**

Part-of-speech (POS) tagging is the task of determining the word class of a token. This is crucial for *disambiguation*, because different parts of speech may have similar forms.

In [ ]:
tagged = nltk.pos_tag(tokens)
tagged

**Removing stop words**

Stop words are English words that do not add much meaning to a sentence. They can safely be ignored without sacrificing the meaning of the sentence. For example, the words like the, he, have etc.

In [ ]:
# setting english stopwords
stopword_list = nltk.corpus.stopwords.words('english')
print(stopword_list)

The above list of stopwords also contains the word "not", and its other forms such as don't, didn't, etc. We need them for correct sentiment classification.

For example, consider a negative review "*not a good movie*", and if we remove 'not' from it then it becomes a positive review "*a good movie*".

In [ ]:
# Exclude 'not' and its other forms from the stopwords list

updated_stopword_list = []

for word in stopword_list:
    if word=='not' or word.endswith("n't"):
        pass
    else:
        updated_stopword_list.append(word)

print(updated_stopword_list)

In [ ]:
# removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    # splitting strings into tokens (list of words)

    # YOUR CODE HERE to create 'tokens' variable by tokenizing 'text'

    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        # filtering out the stop words
        filtered_tokens = [token for token in tokens if token not in updated_stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in updated_stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [ ]:
txt = "The movie was not that great"

# YOUR CODE HERE to call remove_stopwords() on txt

In [ ]:
# apply remove_stopwords function on review column
# YOUR CODE HERE

### Please answer the questions below to complete the experiment:




In [ ]:
#@title What is the primary goal of tokenization in natural language processing? { run: "auto", form-width: "500px", display-mode: "form" }
Answer = "" #@param ["", "To classify text into categories", "To split text into individual units such as words or phrases", "To generate synthetic text data", "To translate text into different languages"]

In [ ]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [ ]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "" #@param {type:"string"}


In [ ]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "" #@param ["","Yes", "No"]


In [ ]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")